<a href="https://colab.research.google.com/github/Itsuki-Hamano123/auto_ml/blob/master/auto-keras/auto_keras_tabledata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [2]:
cd /gdrive/My Drive/機械学習練習/MLモデル構築自動化/auto-keras

/gdrive/My Drive/機械学習練習/MLモデル構築自動化/auto-keras


In [3]:
ls

auto_keras_tabledata.ipynb  structured_data_classifier/  titanic.ipynb


# CSVに出力したテーブルデータでauto-kerasを試す

## パッケージインストール
(6/4 gitのmasterブランチに対してpip install しないと正常に動かない、エラーの原因は修正中の模様
https://github.com/keras-team/autokeras/issues/1095)

In [4]:
!pip install git+https://github.com/keras-team/autokeras.git@master

  Cloning https://github.com/keras-team/autokeras.git (to revision master) to /tmp/pip-req-build-5lgygx8l
  Running command git clone -q https://github.com/keras-team/autokeras.git /tmp/pip-req-build-5lgygx8l
Processing /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43/keras_tuner-1.0.1-cp36-none-any.whl
  Created wheel for autokeras: filename=autokeras-1.0.3-cp36-none-any.whl size=79056 sha256=4d0da6366ab7bb3bce9f40b0de7c79435e259a2dd7588e46bba9114bad251c17
  Stored in directory: /tmp/pip-ephem-wheel-cache-78ptnsnj/wheels/dd/08/a2/e9ffa481d1eb26290b8c2b16ac9db41d3bccc29f9db731bc86
Successfully built autokeras
  Found existing installation: keras-tuner 1.0.0
    Uninstalling keras-tuner-1.0.0:
      Successfully uninstalled keras-tuner-1.0.0
  Found existing installation: autokeras 1.0.2
    Uninstalling autokeras-1.0.2:
      Successfully uninstalled autokeras-1.0.2


In [5]:
# インストールしたautokerasのバージョン確認
!pip show autokeras

Name: autokeras
Version: 1.0.3
Summary: AutoML for deep learning
Home-page: http://autokeras.com
Author: Data Analytics at Texas A&M (DATA) Lab, Keras Team
Author-email: jhfjhfj1@gmail.com
License: MIT
Location: /usr/local/lib/python3.6/dist-packages
Requires: numpy, packaging, tensorflow, keras-tuner, scikit-learn, pandas
Required-by: 


In [6]:
!pip install git+https://github.com/keras-team/keras-tuner.git@master

  Cloning https://github.com/keras-team/keras-tuner.git (to revision master) to /tmp/pip-req-build-8oevz7f4
  Running command git clone -q https://github.com/keras-team/keras-tuner.git /tmp/pip-req-build-8oevz7f4
  Created wheel for keras-tuner: filename=keras_tuner-1.0.0-cp36-none-any.whl size=88924 sha256=b5b4e04b65827e6950b7d7dd2ae69e3ed28706f4fa7ac9f69073a94e66d835b7
  Stored in directory: /tmp/pip-ephem-wheel-cache-wizigv2w/wheels/32/41/eb/bd67480bd69770f9202a237ba92d47ca430d6f92d7540fd87f
Successfully built keras-tuner
ERROR: autokeras 1.0.3 has requirement keras-tuner>=1.0.1, but you'll have keras-tuner 1.0.0 which is incompatible.
  Found existing installation: keras-tuner 1.0.1
    Uninstalling keras-tuner-1.0.1:
      Successfully uninstalled keras-tuner-1.0.1


In [7]:
!pip show keras-tuner

Name: keras-tuner
Version: 1.0.0
Summary: Hypertuner for Keras
Home-page: https://github.com/keras-team/keras-tuner
Author: The Keras Tuner authors
Author-email: kerastuner@google.com
License: Apache License 2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: numpy, colorama, terminaltables, packaging, scikit-learn, tqdm, future, scipy, requests, tabulate
Required-by: autokeras


In [8]:
!pip show tensorflow

Name: tensorflow
Version: 2.2.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: wheel, numpy, termcolor, absl-py, protobuf, gast, wrapt, scipy, google-pasta, keras-preprocessing, h5py, opt-einsum, tensorflow-estimator, grpcio, astunparse, six, tensorboard
Required-by: fancyimpute, autokeras


In [0]:
import os

from sklearn.datasets import load_wine
import autokeras as ak
import tensorflow as tf
import pandas as pd

## モデル定義 + 訓練（探索）

In [0]:
#@form fields
DATA_DIR = '../big_dataset'  #@param {type: "string"}
TRAIN_CSV = 'preprocessed_train.csv'  #@param {type: "string"}
TEST_CSV = 'preprocessed_test.csv'  #@param {type: "string"}

# 予測する列名
CLASS_COLUMN = 'class' #@param {type: "string"}
NUM_CLASSES = 2 #@param {type: "number"}

# 探索するモデルの上限数
MAX_TRIALS = 11  #@param {type: "slider", min: 0, max: 100}
#@markdown ---


In [54]:
load_data = load_wine(return_X_y =True)
load_data

(array([[1.423e+01, 1.710e+00, 2.430e+00, ..., 1.040e+00, 3.920e+00,
         1.065e+03],
        [1.320e+01, 1.780e+00, 2.140e+00, ..., 1.050e+00, 3.400e+00,
         1.050e+03],
        [1.316e+01, 2.360e+00, 2.670e+00, ..., 1.030e+00, 3.170e+00,
         1.185e+03],
        ...,
        [1.327e+01, 4.280e+00, 2.260e+00, ..., 5.900e-01, 1.560e+00,
         8.350e+02],
        [1.317e+01, 2.590e+00, 2.370e+00, ..., 6.000e-01, 1.620e+00,
         8.400e+02],
        [1.413e+01, 4.100e+00, 2.740e+00, ..., 6.100e-01, 1.600e+00,
         5.600e+02]]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2,
 

### モデルのクラス初期化

In [0]:
# 構造化データを元に分類するモデルクラスの初期化
clf = ak.StructuredDataClassifier(max_trials=MAX_TRIALS,# 探索するモデルの上限数
                                  seed=7,
                                  metrics=["accuracy","AUC"]) 

### モデルの訓練（探索）

In [58]:
clf.fit(x=load_data[0],
        y=load_data[1],
        epochs=100, # 各モデルで回すエポック数(default:1000, earlystoppingあり)
        #callbacks=[EarlyStopping(monitor='val_loss',patience=2)], # kerasモデル同様にcallback処理を設定可能
        validation_split=0.2, # 訓練データの末尾から指定した割合が検証データとなる(default:0.2)
        verbose=1, # 通常のkerasモデルで使われるようなキーワード引数も渡せる
        )

Epoch 1/100
5/5 [==============================] - 0s 67ms/step - loss: 114.5369 - accuracy: 0.5188 - auc: 0.5828 - val_loss: 202.5827 - val_accuracy: 0.2037 - val_auc: 0.2500
Epoch 2/100
5/5 [==============================] - 0s 21ms/step - loss: 83.6240 - accuracy: 0.4354 - auc: 0.5828 - val_loss: 168.0761 - val_accuracy: 0.0000e+00 - val_auc: 0.2500
Epoch 3/100
5/5 [==============================] - 0s 15ms/step - loss: 58.2870 - accuracy: 0.4042 - auc: 0.5828 - val_loss: 134.2580 - val_accuracy: 0.0000e+00 - val_auc: 0.2500
Epoch 4/100
5/5 [==============================] - 0s 16ms/step - loss: 33.8346 - accuracy: 0.2438 - auc: 0.5823 - val_loss: 101.0767 - val_accuracy: 0.0000e+00 - val_auc: 0.0000e+00
Epoch 5/100
5/5 [==============================] - 0s 20ms/step - loss: 18.8804 - accuracy: 0.1729 - auc: 0.6094 - val_loss: 89.2839 - val_accuracy: 0.0000e+00 - val_auc: 0.2500
Epoch 6/100
5/5 [==============================] - 0s 15ms/step - loss: 19.1118 - accuracy: 0.2625 - auc:

Epoch 1/100
5/5 [==============================] - 0s 78ms/step - loss: 1.6491 - accuracy: 0.0000e+00 - auc: 0.3909 - val_loss: 1.5675 - val_accuracy: 0.0000e+00 - val_auc: 0.5000
Epoch 2/100
5/5 [==============================] - 0s 16ms/step - loss: 1.5111 - accuracy: 0.0000e+00 - auc: 0.4947 - val_loss: 0.3868 - val_accuracy: 0.0000e+00 - val_auc: 1.0000
Epoch 3/100
5/5 [==============================] - 0s 16ms/step - loss: 1.2995 - accuracy: 0.0000e+00 - auc: 0.5253 - val_loss: 0.4289 - val_accuracy: 0.0000e+00 - val_auc: 1.0000
Epoch 4/100
5/5 [==============================] - 0s 22ms/step - loss: 1.2213 - accuracy: 0.0000e+00 - auc: 0.5636 - val_loss: 3.0522 - val_accuracy: 0.0000e+00 - val_auc: 0.4823
Epoch 5/100
5/5 [==============================] - 0s 16ms/step - loss: 1.2656 - accuracy: 0.0000e+00 - auc: 0.5293 - val_loss: 4.7503 - val_accuracy: 0.0000e+00 - val_auc: 0.3040
Epoch 6/100
5/5 [==============================] - 0s 16ms/step - loss: 1.2414 - accuracy: 0.0000e+0

Epoch 1/100
5/5 [==============================] - 0s 75ms/step - loss: 72.3062 - accuracy: 0.4542 - auc: 0.5214 - val_loss: 66.2351 - val_accuracy: 0.3148 - val_auc: 0.2500
Epoch 2/100
5/5 [==============================] - 0s 20ms/step - loss: 37.1939 - accuracy: 0.3375 - auc: 0.5266 - val_loss: 22.6386 - val_accuracy: 0.0000e+00 - val_auc: 0.2500
Epoch 3/100
5/5 [==============================] - 0s 16ms/step - loss: 6.5326 - accuracy: 0.1542 - auc: 0.6453 - val_loss: 6.6878 - val_accuracy: 0.0000e+00 - val_auc: 0.2639
Epoch 4/100
5/5 [==============================] - 0s 17ms/step - loss: 14.9636 - accuracy: 0.0229 - auc: 0.5782 - val_loss: 20.9271 - val_accuracy: 0.0000e+00 - val_auc: 0.2500
Epoch 5/100
5/5 [==============================] - 0s 22ms/step - loss: 10.9302 - accuracy: 0.0000e+00 - auc: 0.5404 - val_loss: 1.8899 - val_accuracy: 0.0000e+00 - val_auc: 0.5000
Epoch 6/100
5/5 [==============================] - 0s 15ms/step - loss: 8.7364 - accuracy: 0.0000e+00 - auc: 0.45

KeyboardInterrupt: ignored

### モデルの評価

In [0]:
model = clf.export_model()

In [34]:
train_df = pd.read_csv(os.path.join(DATA_DIR, TRAIN_CSV))

train_x = train_df.drop(columns=CLASS_COLUMN).values
train_y = tf.keras.utils.to_categorical(train_df[CLASS_COLUMN].values)
train_y

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       ...,
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.]], dtype=float32)

In [35]:
model.evaluate(train_x, train_y)

UnimplementedError: ignored